In [ ]:
import pandas as pd
import numpy as np
import re

from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pyplot as plt
import seaborn as sns

PATH = '/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/parsed_data/full_dataset.csv'

Mounted at /content/drive


In [ ]:
df=pd.read_csv(PATH)
df.sample(2)

,imones_kodas,imones_pavadinimas,imones_aprasymas,desc_len,EmployeesLast,Nace2,gr_code,gr_stat_letter,gr_stat_code,gr_4digits
24704,302925410,uab smart claims,žalų sureguliavimas nuostolių vertinimas turto...,7,18,662100,9,K,K66,6621
2979,110865943,pruszynski ir partneriai uab,veikla pradejo įmonės pagrindinė veikla plieni...,180,21,467300,44,G,G46,4673


In [ ]:
df = df[df.desc_len<513]
df.shape

(30877, 10)

In [ ]:
df.desc_len.max()

500

# BERT Encodings

In [ ]:
%%capture
!pip install transformers
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

from transformers import AutoModel, AutoTokenizer
model_name = "EMBEDDIA/litlat-bert"

bert_model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/606M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [ ]:
def bert_embeddings(x):
  inputs = tokenizer(x, padding=True, truncation=True, return_tensors="pt", max_length=512)
  outputs = bert_model(**inputs)
  embedings = outputs.pooler_output
  return embedings.detach().numpy()[0]

def labse_embeddings(x):
  embeddings = labse_model.encode(x)
  return embeddings

In [ ]:
# https://metatext.io/models/embeddia-litlat-bert

In [ ]:
bert_embeddings = [bert_embeddings(i) for i in df.imones_aprasymas.tolist()]

In [ ]:
bert = pd.DataFrame(bert_embeddings)
bert.sample(1)

In [ ]:
bert.to_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/parsed_data/bert_embeddings.csv', index=False)

In [ ]:
#df['bert_embeddings'] = df.imones_aprasymas.apply(lambda x: bert_embeddings(x))
#df['labse_embeddings'] = df.imones_aprasymas.apply(lambda x: labse_embeddings(x))

In [ ]:
labse_embeddings = [labse_embeddings(i) for i in df.imones_aprasymas.tolist()]
labse = pd.DataFrame(labse_embeddings)
labse.to_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/parsed_data/labse_embeddings.csv', index=False)

In [ ]:
df.to_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/parsed_data/data_with_embeddings.csv', index=False, sep=';')

In [ ]:
df.dtypes

imones_kodas           int64
imones_pavadinimas    object
imones_aprasymas      object
desc_len               int64
EmployeesLast          int64
Nace2                  int64
gr_code                int64
gr_stat_letter        object
gr_stat_code          object
gr_4digits             int64
bert_embeddings       object
labse_embeddings      object
dtype: object

In [ ]:
type(df.at[0, 'bert_embeddings'])

numpy.ndarray